In [1]:
import numpy as np
import pandas as pd
import sklearn

import os
os.environ["KERAS_BACKEND"] = "tensorflow"

import keras
print(keras.__version__)

2024-08-14 05:53:00.247927: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-14 05:53:00.250571: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-14 05:53:00.258105: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-14 05:53:00.270203: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-14 05:53:00.273595: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-14 05:53:00.283013: I tensorflow/core/platform/cpu_feature_gu

3.4.1


In [2]:
import glob
dirname = './data/train'
filenames = glob.glob(dirname+'/*')
rows = []
for filename in filenames:
    file = open(filename, 'r')
    lines = file.readlines()
    for line in lines:
        fname = filename.split('/')[-1].split('.')[0]
        name = line.strip().replace(u'\xa0', u' ').replace(' ', '').replace('-', '').replace("'", '').replace(':', '').replace('/', '').replace(',', '').replace('1', '').lower()
        rows.append({'Name': name, 'Origin': fname})
    file.close()
df = pd.DataFrame(rows)

In [3]:
test = pd.read_csv('./data/test.csv')
test['Name'] = test['Name'].apply(lambda x: x.replace(u'\xa0', u' ').replace(' ', '').replace('-', '').replace("'", '').replace(':', '').replace('/', '').replace(',', '').replace('1', '').lower())

In [4]:
chars = set()
for name in df['Name']:
    chars.update(name.lower())
chars = sorted(list(chars))

In [5]:
longest_string = sorted(df['Name'], key=lambda x: len(x))[-1]
max_len = len(longest_string)
max_len

19

In [6]:
def encode(my_word, chars, max_len):
    raw = np.zeros((max_len, len(chars)))
    for i in range(len(my_word)):
        raw[i][chars.index(my_word[i])] = 1
    return raw

In [7]:
X = np.array([encode(word, chars, max_len) for word in df['Name']])

In [8]:
test_x = np.array([encode(word, chars, max_len) for word in test['Name']])

In [9]:
tmp = sorted(list(df['Origin'].unique()))
df['Origin'] = df['Origin'].apply(lambda x: tmp.index(x))
y = df['Origin']

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.1)

In [11]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, SimpleRNN, Dropout, Input, LSTM, BatchNormalization, GRU
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy

In [12]:
X_train.shape, y_train.shape

((18066, 19, 50), (18066,))

In [13]:
model = Sequential([
    Input(shape=(max_len, len(chars))),
    GRU(256, return_sequences=False),
    Flatten(),
    Dense(18, activation='softmax'),
])
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [14]:
epochs = 120
batch_size = 1024
history = model.fit(X_train, y_train,
                    epochs = epochs,
                    batch_size = batch_size,
                    validation_data=(X_valid,y_valid))

Epoch 1/120
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 196ms/step - accuracy: 0.3881 - loss: 2.6771 - val_accuracy: 0.4691 - val_loss: 2.0585
Epoch 2/120
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 184ms/step - accuracy: 0.4696 - loss: 1.9362 - val_accuracy: 0.4691 - val_loss: 1.7929
Epoch 3/120
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 184ms/step - accuracy: 0.4681 - loss: 1.8204 - val_accuracy: 0.4691 - val_loss: 1.7604
Epoch 4/120
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 185ms/step - accuracy: 0.4691 - loss: 1.7770 - val_accuracy: 0.4691 - val_loss: 1.7118
Epoch 5/120
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 183ms/step - accuracy: 0.4666 - loss: 1.7111 - val_accuracy: 0.4691 - val_loss: 1.6486
Epoch 6/120
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 184ms/step - accuracy: 0.4726 - loss: 1.6297 - val_accuracy: 0.4691 - val_loss: 1.6107
Epoch 7/120
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 185ms/step - accuracy: 0.4734 - loss: 1.5980 - val_accuracy: 0.4875 - val_loss: 1.5754
Epoch 8/120
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 182ms/step - accuracy: 0.5092 - loss: 1.5806 - val_accu

KeyboardInterrupt: 

In [ ]:
from matplotlib import pyplot as plt

plt.plot(history.history['loss'], 'r')
plt.plot(history.history['val_loss'], 'b')
plt.ylabel('AUC')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
y_pred = model.predict(test_x)

In [ ]:
submission = pd.DataFrame({'prediction': np.array([np.argmax(i) for i in y_pred])})
submission